In [1]:
import os
import numpy as np
import matplotlib.pyplot as plt
from PIL import Image, ImageOps
import random
import torch
import torchvision
import torchvision.transforms as transforms
import torchvision.datasets as datasets
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
from torch.autograd import Variable
from torch.utils.data import DataLoader

## Set Configs

#### NOTE: You may need to change the path to the data folder.

In [2]:
TRAIN_DIR = 'C:\\Users\\HP\\Desktop\\one-shot-steel-surfaces-master\\dataset_all\\cnn_data\\training'
TEST_DIR = 'C:\\Users\\HP\\Desktop\\one-shot-steel-surfaces-master\\dataset_all\\cnn_data\\testing'

WEIGHT_PATH = './weights/steel/cnn/'
SAVE_IMAGE_PATH = './images/steel/cnn/'

SAVE_TEST_RESULTS_PATH = './images/steel/test_output'

device = torch.device('cuda:0' if (torch.cuda.is_available()) else 'cpu')

## Parameters

In [3]:
BATCH_SIZE = 128
N_EPOCHS = 80
LR = 0.0005
N_GPU = 1
SET_L = False
INPUT_D = 3 if SET_L == False else 1

## CNN Architecture

In [4]:
class CNNNet(nn.Module):
    def __init__(self):
        super(CNNNet, self).__init__()
        self.conv_layer = nn.Sequential(
            nn.ZeroPad2d(1),
            nn.Conv2d(INPUT_D, 4, kernel_size=3),
            nn.ReLU(),        
            nn.ZeroPad2d(1),
            nn.Conv2d(4, 8, kernel_size=3),
            nn.ReLU(),
            nn.ZeroPad2d(1),
            nn.Conv2d(8, 8, kernel_size=3),
            nn.ReLU(),
        )
        
        self.fc_layer = nn.Sequential(
            nn.Linear(8*100*100, 500),
            nn.ReLU(),
            nn.Linear(500, 500),
            nn.ReLU(),
            nn.Linear(500, 6),
        )
        
    def forward(self, x):
        x = self.conv_layer(x)
        x = x.view(x.size()[0], -1)
        x = self.fc_layer(x)
        return x

## Set Helpers

In [5]:
def show_img(img, text=None, figsize=(20,20), save_into=None):
    img = img.numpy()
    plt.figure(figsize=figsize)
    plt.axis('off')
    if text:
        plt.text(50, 8, text, bbox={'facecolor': 'white', 'alpha': 1 })
    plt.imshow(np.transpose(img, (1, 2, 0)))
    
    if save_into:
        plt.savefig(save_into)
        plt.show()
    else:
        plt.show()

In [6]:
def explain_databatch(batch):
    """
    batch: tensor batch from dataset
    """
    print('Each batch of training data is a tuple of {} elements.'.format(len(batch)))
    
    print('Shape of each element of a tuple in the batch:')
    for i in range(len(batch)):
        print('element#{}: {}'.format(i, batch[i].shape))

## Configure Custom Data Loader

In [7]:
class RandRotateTransform:
    """Rotate by one of the given angles."""

    def __init__(self, angles):
        self.angles = angles

    def __call__(self, x):
        angle = random.choice(self.angles)
        return transforms.functional.rotate(x, angle)


In [8]:
transform = transforms.Compose([
                            transforms.Resize((100,100)),
                            RandRotateTransform(angles=[0, 90, 180, 270]),
                            transforms.RandomHorizontalFlip(),
                            transforms.ToTensor(),
                            transforms.Normalize((0.5,), (0.5,))
])

In [9]:
trainset = torchvision.datasets.ImageFolder(root=TRAIN_DIR, transform=transform)
train_loader = torch.utils.data.DataLoader(trainset, batch_size=BATCH_SIZE,
                                          shuffle=True, num_workers=0)

## Prepare Training Data

In [10]:
example_batch = next(iter(train_loader))

explain_databatch(example_batch)

Each batch of training data is a tuple of 2 elements.
Shape of each element of a tuple in the batch:
element#0: torch.Size([128, 3, 100, 100])
element#1: torch.Size([128])


In [11]:
model = CNNNet().to(device)
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=LR)

In [12]:
for epoch in range(120):
    running_loss = 0.0
    for i, data in enumerate(train_loader, 0):
        inputs, labels = data
        inputs, labels = inputs.to(device), labels.to(device)

        optimizer.zero_grad()

        # forward + backward + optimize
        outputs = model(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
        
        running_loss += loss.item()
        
        if i % 10 == 0:
            running_loss = 0.0
    print('[%d] \t loss: %.6f' % (epoch + 1, running_loss / 2000))

print('\n\nFinished Training!!')

[1] 	 loss: 0.000758
[2] 	 loss: 0.000469
[3] 	 loss: 0.000563
[4] 	 loss: 0.000554
[5] 	 loss: 0.000454
[6] 	 loss: 0.000488
[7] 	 loss: 0.000346
[8] 	 loss: 0.000352
[9] 	 loss: 0.000391
[10] 	 loss: 0.000286
[11] 	 loss: 0.000318
[12] 	 loss: 0.000369
[13] 	 loss: 0.000530
[14] 	 loss: 0.000158
[15] 	 loss: 0.000226
[16] 	 loss: 0.000162
[17] 	 loss: 0.000202
[18] 	 loss: 0.000112
[19] 	 loss: 0.000106
[20] 	 loss: 0.000189
[21] 	 loss: 0.000228
[22] 	 loss: 0.000228
[23] 	 loss: 0.000085
[24] 	 loss: 0.000065
[25] 	 loss: 0.000040
[26] 	 loss: 0.000066
[27] 	 loss: 0.000041
[28] 	 loss: 0.000061
[29] 	 loss: 0.000087
[30] 	 loss: 0.000067
[31] 	 loss: 0.000192
[32] 	 loss: 0.000140
[33] 	 loss: 0.000045
[34] 	 loss: 0.000129
[35] 	 loss: 0.000008
[36] 	 loss: 0.000092
[37] 	 loss: 0.000030
[38] 	 loss: 0.000095
[39] 	 loss: 0.000086
[40] 	 loss: 0.000051
[41] 	 loss: 0.000050
[42] 	 loss: 0.000063
[43] 	 loss: 0.000022
[44] 	 loss: 0.000025
[45] 	 loss: 0.000023
[46] 	 loss: 0.0000

### Test the trained model

In [13]:
torch.save(model.state_dict(), 'C:\\Users\\HP\\Desktop\\one-shot-steel-surfaces-master\\test_cnnnet.pth')
testset = torchvision.datasets.ImageFolder(root=TEST_DIR, transform=transform)
testloader = torch.utils.data.DataLoader(testset, batch_size=BATCH_SIZE, shuffle=False, num_workers=0)

In [14]:

correct = 0
total = 0
rs=0
with torch.no_grad():
    for data in testloader:
        images, labels = data
        images, labels = images.to(device), labels.to(device)
        outputs = model(images)
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()
        

print('Accuracy of the network on the test images: %d %%' % (100 * correct / total))


Accuracy of the network on the test images: 91 %
